# Experiment 1: Not sure what to call this (False Goal Test?)
The different ToM models may struggle with learning which goals are relevant to which agents. For example, an agent wants to go to point B. The path the agent takes will go by point A, but historically, this agent does not visit point A. How well do models learn goal relevance for agents?  

We can test this by designing specific paths for the agents. We will choose the highest probability goal for each agent and the lowest probability goal. Then, we will for the agent to start from a position before the low probability goal such that this goal has to be passed on the way to the true goal. As the agent approaches the false goal, are the models fooled with high certainty that this is the correct goal driving the agent?  

Steps:
1. Load agents and data from experiment 0
2. Choose the starting and goal nodes for each agent
3. Run an experiment for each agent (100) using the models trained on experiment 0 data.
4. Visualize the performance

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pymc
import networkx as nx
import matplotlib.pyplot as plt
import random
import json
import torch

In [3]:
import sys
import os
import osmnx as ox

# Adjust this path as needed to point to your project root
sys.path.append(os.path.abspath(".."))

In [4]:
from real_world_src.environment.campus_env import CampusEnvironment
from real_world_src.agents.agent_factory import AgentFactory
from real_world_src.agents.agent_species import ShortestPathAgent
from real_world_src.simulation.simulator import Simulator
#from real_world_src.simulation.experiment_1 import Simulator

from real_world_src.utils.run_manager import RunManager
from real_world_src.utils.config import VISUAL_CONFIG
from real_world_src.utils.config import get_agent_color

In [5]:
def visualize_map(campus,
                  path=None,
                  max_goal=None,
                  min_goal=None, 
                  start_goal=None,
                  ax=None):
    """
    Visualize the campus map, plus optional trajectory, start, waypoint, and goal nodes.
    
    Arguments:
      campus         an object with:
                        - campus.G           : a networkx graph
                        - campus.node_coords : dict[node] -> (x, y) coordinates
                        - campus.buildings   : a GeoDataFrame of building footprints (or None)
      path           list of node‐IDs, the agent’s trajectory to draw (in order)
      start_nodes    list of node‐IDs to mark as start(s)
      waypoint       single node‐ID to mark as an intermediate waypoint
      goal_nodes     list of node‐IDs to mark as goal(s)
      ax             optional matplotlib Axes onto which to draw
      
    Returns:
      fig, ax        the figure & axis
    """
    # 1) draw the base map
    if ax is None:
        fig, ax = ox.plot_graph(campus.G,
                                figsize=(15,15),
                                node_size=5,
                                edge_color="#999999",
                                show=False,
                                close=False)
    else:
        fig = ax.figure
        ox.plot_graph(campus.G,
                      ax=ax,
                      node_size=5,
                      edge_color="#999999",
                      show=False)
    
    # 2) overlay buildings if you have them
    if getattr(campus, "buildings", None) is not None:
        campus.buildings.plot(ax=ax,
                              color="#bbbbbb",
                              alpha=0.7,
                              edgecolor="#444444")
    
    # 3) draw the agent’s path
    if path is not None and len(path) > 1:
        coords = [campus.node_coords[n] for n in path]
        xs, ys = zip(*coords)
        ax.plot(xs, ys,
                linestyle='-',
                linewidth=3.0,
                color='lightblue',
                alpha=0.8,
                zorder=3,
                label="Trajectory")
        # and mark the actual visited nodes
        ax.scatter(xs, ys,
                   c='C0',
                   s=30,
                   zorder=4)
    
    # 6) mark goal nodes
    if max_goal:
        goal_coords = [campus.node_coords[max_goal]]
        xs, ys = zip(*goal_coords)
        ax.scatter(xs, ys,
                   c="yellow",
                   s=160,
                   marker="*",
                   zorder=5,
                   label="Max Goal")

    if min_goal:
        goal_coords = [campus.node_coords[min_goal]]
        xs, ys = zip(*goal_coords)
        ax.scatter(xs, ys,
                   c="red",
                   s=160,
                   marker="*",
                   zorder=5,
                   label="Min Goal")

    if start_goal:
        goal_coords = [campus.node_coords[start_goal]]
        xs, ys = zip(*goal_coords)
        ax.scatter(xs, ys,
                   c="lightgreen",
                   s=160,
                   marker="*",
                   zorder=5,
                   label="Start Node")
    
    # 7) finalize
    ax.legend(loc="upper right")
    ax.set_title("UCSD Campus Environment")
    ax.set_axis_off()
    fig.tight_layout()
    return fig, ax

## Step 1: Loading Data

In [6]:
# Create a run manager
run_manager = RunManager('visuals')
run_dir = run_manager.start_new_run()

# Initialize campus environment
campus = CampusEnvironment()

Started new simulation run #21
All visualizations will be saved to: /Users/arshia/UCSD/Theory-of-mind/notebooks/visuals/run_21_20250718_021131
Loading map data for University of California, San Diego, La Jolla, CA, USA...
Environment loaded with 3136 nodes and 8704 edges


In [7]:
# Need to establish the set of common goals (just choose the landmark nodes)
goals = [469084068, 49150691, 768264666, 1926666015, 1926673385, 49309735,
         273627682, 445989107, 445992528, 446128310, 1772230346, 1926673336, 
         2872424923, 3139419286, 4037576308]

In [8]:
import pickle
# if you used dill, just replace pickle with dill

with open('./../data/agents.pkl', 'rb') as f:
    agents = pickle.load(f)

/var/folders/xg/fld01xsn2c1bbnbsz7mzrg6r0000gn/T/ipykernel_34282/3168561831.py:5: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  agents = pickle.load(f)


In [9]:
with open("./../data/path_data.json", 'r') as file:
    path_data = json.load(file)

with open("./../data/goal_data.json", 'r') as file:
    goal_data = json.load(file)

In [10]:
def convert_keys_to_int(data):
    if isinstance(data, dict):
        return {int(k) if isinstance(k, str) and k.isdigit() else k: convert_keys_to_int(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_keys_to_int(item) for item in data]
    else:
        return data

In [11]:
goal_data = convert_keys_to_int(goal_data)
path_data = convert_keys_to_int(path_data)

## Step 2: Choosing Start and Goal

In [12]:
from tqdm import tqdm

In [12]:
all_nodes = set()
for episode in path_data.values():
    for path in episode.values():
        if isinstance(path, (list, tuple, set)):
            all_nodes.update(path)
        else:
            all_nodes.add(path)
all_nodes.update(campus.G_undirected.nodes())


to_remove = []
for agent in tqdm(agents, desc="Processing agents"):
    max_goal_idx = np.argmax(agent.goal_distribution)
    max_goal = goals[max_goal_idx]
    
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]

    #G = campus.G_undirected  # your undirected NetworkX graph
    G = campus.G 
    
    candidates = []
    #for s in G.nodes():
    for s in all_nodes:
        if s not in G.nodes():
            continue
        if s in (min_goal, max_goal):
            continue  # optional: don’t start right on one of the goals
        try:
            path = nx.shortest_path(G, s, max_goal, weight="length")
        except nx.NetworkXNoPath:
            continue
        if min_goal in path:
            candidates.append(s)
    
    if not candidates:
        print("No start node forces a pass by min_goal on the way to max_goal")
        to_remove.append(agent)
        continue

    agent.goal_node = max_goal
    agent.start_node = random.choice(candidates)
    agent.current_node = agent.start_node
    agent.plan_path()

    if min_goal not in agent.path:
        print("Problem, missing min goal in path")

Processing agents:   9%|▉         | 9/100 [01:03<12:58,  8.56s/it]

Problem, missing min goal in path


Processing agents:  17%|█▋        | 17/100 [02:01<10:01,  7.25s/it]

No start node forces a pass by min_goal on the way to max_goal


Processing agents:  18%|█▊        | 18/100 [02:10<10:28,  7.67s/it]

...

Problem, missing min goal in path


Processing agents:  94%|█████████▍| 94/100 [12:27<00:46,  7.81s/it]

No start node forces a pass by min_goal on the way to max_goal


Processing agents: 100%|██████████| 100/100 [13:06<00:00,  7.87s/it]

Problem, missing min goal in path


In [13]:
agents = [agent for agent in agents if agent not in to_remove]

In [33]:
print(run_dir)

/Users/arshia/UCSD/Theory-of-mind/notebooks/visuals/run_20_20250718_014549


In [35]:
with open('./../data/agents_processed.pkl', 'wb') as f:
    pickle.dump(agents, f)

In [13]:
with open('./../data/agents_processed.pkl', 'rb') as f:
    agents = pickle.load(f)

In [14]:
# Get each agent's path 
for agent in agents:
    agent.plan_path()

## Step 3: Run the Models

### BToM

In [15]:
# Import BToM baseline
from real_world_src.models.btom import BToM

In [16]:
btom = BToM(campus, agents, goals)

Computing shortest paths...
Done


In [17]:
# Calculate all the posteriors using the first 30 episode
posteriors_data = {}

In [18]:
for agent in agents:
    posteriors_data[agent.id] = btom.update_agent_posterior_over_path(agent, agent.path)
# Reset the posteriors for new episode
btom.reset_posteriors()

NetworkXError: The node 1352179344 is not in the digraph.

### Extended BToM

In [ ]:
from real_world_src.models.extended_btom import Extended_BToM

In [ ]:
with open('extended_btom.pkl', 'rb') as f:
    ext_btom = pickle.load(f)

In [ ]:
# Calculate all the posteriors using the first 30 episode
ext_posteriors_data = {}

for agent in agents:
    ext_posteriors_data[agent.id] = ext_btom.update_agent_posterior_over_path(agent, agent.path)


# LSTM

In [ ]:
from real_world_src.models.lstm import (
    SimpleLSTM, LSTMDataset, lstm_collate_fn,
    train_lstm_model, evaluate_lstm_model, predict_goal_posterior
)
from torch.utils.data import DataLoader

In [ ]:
# build node2idx so that every node in campus.G_undirected maps to 0…V−1
all_nodes = list(campus.G_undirected.nodes())
node2idx  = {n:i for i,n in enumerate(all_nodes)}
V = len(all_nodes)

# build goal2idx likewise for your goals list
goal2idx = {g:i for i,g in enumerate(goals)}
G = len(goals)

In [ ]:
lstm_dataset = LSTMDataset(path_data, goal_data, node2idx, goal2idx)
lstm_loader = DataLoader(lstm_dataset, batch_size=32, shuffle=True, collate_fn=lstm_collate_fn)

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
lstm_model = SimpleLSTM(num_nodes=len(node2idx), num_goals=len(goal2idx)).to(device)
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=1e-3)
train_lstm_model(lstm_model, lstm_loader, optimizer, device, num_epochs=10)

In [ ]:
evaluate_lstm_model(lstm_model, lstm_loader, device)

In [ ]:
n_test = int(0.2 * len(agents))
test_agent_ids = [agent.id for agent in agents[-n_test:]]

In [ ]:
# Compute posteriors along paths for all test agents (LSTM)
lstm_posteriors = {}
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    path = agent.path
    posteriors_along_path = []
    for t in range(1, len(path)+1):
        subpath = path[:t]
        posterior = predict_goal_posterior(lstm_model, subpath, node2idx, goal2idx, device)
        posteriors_along_path.append(posterior)
    lstm_posteriors[agent.id] = posteriors_along_path

# GRU

In [ ]:
from real_world_src.models.gru import (
    SimpleGRU, GRUDataset, gru_collate_fn,
    train_gru_model, evaluate_gru_model, predict_goal_posterior_gru
)
from torch.utils.data import DataLoader

In [ ]:
gru_dataset = GRUDataset(path_data, goal_data, node2idx, goal2idx)
gru_loader = DataLoader(gru_dataset, batch_size=32, shuffle=True, collate_fn=gru_collate_fn)

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
gru_model = SimpleGRU(num_nodes=len(node2idx), num_goals=len(goal2idx)).to(device)
optimizer = torch.optim.Adam(gru_model.parameters(), lr=1e-3)
train_gru_model(gru_model, gru_loader, optimizer, device, num_epochs=10)

In [ ]:
evaluate_gru_model(gru_model, gru_loader, device)

In [ ]:
from tqdm import tqdm

In [ ]:
# Compute posteriors along paths for all test agents(GRU)
gru_posteriors = {}
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    path = agent.path
    posteriors_along_path = []
    for t in range(1, len(path)+1):
        subpath = path[:t]
        posterior = predict_goal_posterior_gru(gru_model, subpath, node2idx, goal2idx, device)
        posteriors_along_path.append(posterior)
    gru_posteriors[agent.id] = posteriors_along_path

### ToMNet

In [ ]:
import torch
from real_world_src.models.simple_tomnet import ToMNet, ToMNetDataset, tomnet_collate_fn, infer_goal_dists, make_support_tensor
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = 'mps'

# the values you trained with:
K     = 10
T_sup = 75
T_q   = 20


model = ToMNet(
    num_nodes = len(campus.nodes), # to fix the error in checkpoint
    #num_nodes=3108,
    num_goals = len(goals),
    K     = K,
    T_sup = T_sup,
    T_q   = T_q,
).to(device)

state_dict = torch.load(
    "data/All_en_tomnet_cuda_1.pth",  # Path to the saved model checkpoint
    map_location=device
)
model.load_state_dict(state_dict)
model.eval()

In [ ]:
# build node2idx so that every node in campus.G_undirected maps to 0…V−1
all_nodes = list(campus.G_undirected.nodes())
node2idx  = {n:i for i,n in enumerate(all_nodes)}
V = len(all_nodes)

# build goal2idx likewise for your goals list
goal2idx = {g:i for i,g in enumerate(goals)}
G = len(goals)

In [ ]:
train_agent_ids = list(range(0, 70))
test_agent_ids = list(range(70, 100))

In [ ]:
def get_posterior_format(dists):
    idx2goal = { idx: goal for goal, idx in goal2idx.items() }
    goal_posterior = [ {idx2goal[i]: float(p) for i, p in enumerate(prob_row) } for prob_row in dists]
    return goal_posterior

In [ ]:
import torch.nn.functional as F


In [ ]:
def infer_single_path_goal_dist(
    model, agent_id, test_ep, agent_path,
    path_data, node2idx, goal2idx,
    K, T_sup, T_q,
    device='cuda'
):
    model.eval()
    # 1) build support once
    sup     = make_support_tensor(agent_id, test_ep, path_data, node2idx, K, T_sup)
    sup     = sup.to(device).unsqueeze(0)     # add batch‐dim → [1,K,T_sup]

    raw_seq = agent_path
    idxs    = [node2idx[n] for n in raw_seq]
    N       = len(idxs)

    goal_dists = []   # will be list of length N each [num_goals]
    with torch.no_grad():
        for t in range(1, N):
            # build prefix up to t (we treat t=0 as “no steps seen”)
            prefix_len = min(t, T_q)
            # pad prefix to T_q
            prefix = torch.zeros(T_q, dtype=torch.long)
            if prefix_len>0:
                prefix[:prefix_len] = torch.tensor(idxs[:prefix_len], dtype=torch.long)
            # move to device and batch‐dim
            prefix     = prefix.to(device).unsqueeze(0)       # [1,T_q]
            prefix_len = torch.tensor([prefix_len], dtype=torch.long, device=device)

            # forward through ToMNet
            _, goal_logits = model(sup, prefix, prefix_len)   # [1, num_goals]
            p_goal = F.softmax(goal_logits, dim=-1)[0]        # remove batch‐dim → [num_goals]

            goal_dists.append(p_goal.cpu().numpy())

    return goal_dists   # shape (N × num_goals) array

In [ ]:
tomnet_posteriors = {}
episode=0

for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    dists = infer_single_path_goal_dist(model, agent.id, episode, agent.path, path_data, node2idx, goal2idx, K=10, T_sup=75, T_q=20, device=device)
    #dists = infer_goal_dists(model, agent.id, episode, path_data, node2idx, goal2idx, K=10, T_sup=75, T_q=20, device=device)
    tomnet_posteriors[agent.id] = get_posterior_format(dists)

## Step 4: Calculate Brier Score  
We will calculate the brier score for each 10% portion of the path (same as before), but only leading up to the false goal. A model that does not learn historical trends will have a higher brier score since it will be more confident in the false goal. 

In [ ]:
from real_world_src.utils.metrics import brier_along_path, accuracy_along_path, fooled_along_path

In [ ]:
btom_scores = []
for agent in agents:
    path = agent.path
    btom_scores.append(brier_along_path(path, 
                              agent.goal_node, 
                              posteriors_data[agent.id], 
                              goals))

ext_scores = []
for agent in agents:
    path = agent.path
    ext_scores.append(brier_along_path(path, 
                              agent.goal_node, 
                              ext_posteriors_data[agent.id], 
                              goals))

tomnet_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    path = agent.path
    tomnet_scores.append(brier_along_path(path, 
                              agent.goal_node, 
                              tomnet_posteriors[agent.id], 
                              goals))
    
lstm_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    path = agent.path
    lstm_scores.append(brier_along_path(path, 
                              agent.goal_node, 
                              lstm_posteriors[agent.id], 
                              goals))

    
gru_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    path = agent.path
    gru_scores.append(brier_along_path(path, 
                              agent.goal_node, 
                              gru_posteriors[agent.id], 
                              goals))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Suppose `scores` and `ext_scores` are your (N × 11) arrays:
arr      = np.array(btom_scores)      # shape (N, 11)
ext_arr  = np.array(ext_scores)  # same shape
tomnet_arr = np.array(tomnet_scores)
lstm_arr= np.array(lstm_scores)
gru_arr= np.array(gru_scores)
props = np.linspace(0, 1, arr.shape[1])


# Compute means 
mean_b = arr.mean(axis=0)
mean_e = ext_arr.mean(axis=0)
mean_tomnet = tomnet_arr.mean(axis=0)
mean_lstm = lstm_arr.mean(axis=0)
mean_gru = gru_arr.mean(axis=0)


plt.figure(figsize=(10,6))

# BToM curve
plt.plot(props, mean_b, '-o', color='C0', label='BToM mean')
# Extended BToM curve
plt.plot(props, mean_e, '-s', color='C1', label='Ext-BToM mean')
# Tomnet curve
plt.plot(props, mean_tomnet, '-o', color='green', label='ToMNet')
#LSTM curve
plt.plot(props, mean_lstm, '-^', color='purple', label='LSTM')
#GRU curve
plt.plot(props, mean_gru, '-d', color='orange', label='GRU')


plt.xlabel('Proportion of Path Observed')
plt.ylabel('Brier Score')
plt.title('Brier Score vs. Path Fraction (with 95% CI)')
plt.xticks(props)
plt.ylim(0, 1.7)
plt.grid(alpha=0.3)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
fooled_scores = []
for agent in agents:
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]
    # Select the path proportion 
    path = agent.path
    false_goal_idx = path.index(min_goal)
    path = path[:false_goal_idx + 1]
    
    fooled_scores.append(fooled_along_path(path, 
                                    agent.goal_node,
                                    min_goal,
                                    posteriors_data[agent.id],
                                    goals))

ext_fooled_scores = []
for agent in agents:
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]
    # Select the path proportion 
    path = agent.path
    false_goal_idx = path.index(min_goal)
    path = path[:false_goal_idx + 1]
    
    ext_fooled_scores.append(fooled_along_path(path, 
                                    agent.goal_node,
                                    min_goal,
                                    ext_posteriors_data[agent.id],
                                    goals))

tomnet_fooled_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]
    # Select the path proportion 
    path = agent.path
    false_goal_idx = path.index(min_goal)
    path = path[:false_goal_idx + 1]
    
    tomnet_fooled_scores.append(fooled_along_path(path, 
                                    agent.goal_node,
                                    min_goal,
                                    tomnet_posteriors[agent.id],
                                    goals))
    
lstm_fooled_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]
    # Select the path proportion 
    path = agent.path
    false_goal_idx = path.index(min_goal)
    path = path[:false_goal_idx + 1]
    
    lstm_fooled_scores.append(fooled_along_path(path, 
                                    agent.goal_node,
                                    min_goal,
                                    lstm_posteriors[agent.id],
                                    goals))
gru_fooled_scores = []
for agent in agents:
    if agent.id not in test_agent_ids:
        continue
    min_goal_idx = np.argmin(agent.goal_distribution)
    min_goal = goals[min_goal_idx]
    # Select the path proportion 
    path = agent.path
    false_goal_idx = path.index(min_goal)
    path = path[:false_goal_idx + 1]
    
    gru_fooled_scores.append(fooled_along_path(path, 
                                    agent.goal_node,
                                    min_goal,
                                    gru_posteriors[agent.id],
                                    goals))
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# your two (n_trajectories × n_fractions) arrays:
arr      = np.array(fooled_scores)      # original BToM
ext_arr  = np.array(ext_fooled_scores)  # extended BToM
tomnet_arr = np.array(tomnet_fooled_scores)
lstm_arr= np.array(lstm_fooled_scores)
gru_arr= np.array(gru_fooled_scores)

props = np.linspace(0, 1, arr.shape[1])

# --- original BToM stats ---
mean_b = arr.mean(axis=0)

# --- extended BToM stats ---
mean_e = ext_arr.mean(axis=0)

mean_tomnet = tomnet_arr.mean(axis=0)
mean_lstm = lstm_arr.mean(axis=0)
mean_gru = gru_arr.mean(axis=0)
# Plotting the results


plt.figure(figsize=(10,6))

# original BToM
plt.plot(props, mean_b, '-o', color='C0', label='BToM mean')

# extended BToM
plt.plot(props, mean_e, '-s', color='C1', label='Ext‐BToM mean')
# Tomnet curve
plt.plot(props, mean_tomnet, '-o', color='green', label='ToMNet')
#LSTM curve
plt.plot(props, mean_lstm, '-^', color='purple', label='LSTM')
#GRU curve
plt.plot(props, mean_gru, '-d', color='orange', label='GRU')



plt.xlabel('Proportion of Path Observed Before False Goal')
plt.ylabel('P(false goal)')
plt.title('Model False Goal Confidence')
plt.xticks(props)
plt.ylim(0, 0.5)
plt.grid(alpha=0.3)
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()